In [4]:
import data_read.Read_TFRecords_fcv as read
import data_read.Read_TFRecords as read_aug
from util_LOSS import *
from util_GRAD import *
from util_METRIC import *
from util_ETC import *
import random
import numpy as np
from tensorflow.python.ops import control_flow_ops
import matplotlib.pyplot as plt

In [5]:
class Confi_Net(object):
    def __init__(self, config):
        self.cf=config
        
        # placeholder for training
        self.img_b=tf.placeholder(tf.float32, [None, self.cf.height, self.cf.width, 3])  # 240x320
        self.dpt_b=tf.placeholder(tf.float32, [None, self.cf.height, self.cf.width, 1])   # 240x320
        self.is_train=tf.placeholder(tf.bool)
        
        # placeholder for noise
        #self.noise = tf.placeholder(tf.float32, [None, int(self.cf.height/8)-2, int(self.cf.width/8)-2, 1])
        
        # resizing for calculating
        self.dpt_r1 = tf.image.resize_images(self.dpt_b,  [int(self.cf.height/8), int(self.cf.width/8)])  # 30x40
        self.dpt_r2 = tf.image.resize_images(self.dpt_b,  [int(self.cf.height/4), int(self.cf.width/4)])
        self.dpt_r3 = tf.image.resize_images(self.dpt_b,  [int(self.cf.height/2), int(self.cf.width/2)])
        self.img_r = tf.image.resize_images(self.img_b, [int(self.cf.height/8), int(self.cf.width/8)])
        
        # inference
        self.infer, self.confi1, self.confi2, self.confi3 = self.network(self.vgg(self.img_b), self.img_b, reuse=False, is_train=self.is_train)  
        self.infer_size = tf.image.resize_images(self.infer, [int(self.cf.height/1), int(self.cf.width/1)]) # 40x30 -> 80x60  # for save image
        
        # infersize
        self.infer_r1 = tf.image.resize_images(self.infer,  [int(self.cf.height/8), int(self.cf.width/8)])  # 30x40
        self.infer_r2= tf.image.resize_images(self.infer,  [int(self.cf.height/4), int(self.cf.width/4)])
        
        # confidence map
        self.confimap_1= tf.maximum(self.confimap(self.confi1), 1 / ((self.cf.height/8)*(self.cf.width/8))) 
        self.confimap_2= tf.maximum(self.confimap(self.confi2), 1 / ((self.cf.height/4)*(self.cf.width/4))) 
        self.confimap_3= tf.maximum(self.confimap(self.confi3), 1 / ((self.cf.height/2)*(self.cf.width/2))) 
       
        # resize
        self.confimap_1_size = tf.image.resize_images(self.confimap_1, [int(self.cf.height/4), int(self.cf.width/4)]) # 40x30 -> 80x60  # for save image
        self.confimap_2_size = tf.image.resize_images(self.confimap_2, [int(self.cf.height/2), int(self.cf.width/2)]) # 
        self.confimap_3_size = tf.image.resize_images(self.confimap_3, [int(self.cf.height/1), int(self.cf.width/1)]) # 

        # variance
        _, self.var_1 = tf.nn.moments(self.confimap_1, axes=[1, 2])
        _, self.var_2 = tf.nn.moments(self.confimap_2, axes=[1, 2])
        _, self.var_3 = tf.nn.moments(self.confimap_3, axes=[1, 2])
        self.r_var_1 = tf.reduce_mean(self.var_1)
        self.r_var_2 = tf.reduce_mean(self.var_2)
        self.r_var_3 = tf.reduce_mean(self.var_3)

        
        # Define Cost
        self.l2_cost = L2_img(self.infer, self.dpt_r3)
        self.str_cost = strLoss(self.infer, self.dpt_r3, self.cf)
        self.scale_inv_cost = scaleInvLoss(self.infer, self.dpt_r3, self.cf)
        self.confi1_cost = confiStrLoss(self.infer_r1, self.dpt_r1, self.confimap_1, self.var_1, self.cf)
        self.confi2_cost = confiStrLoss(self.infer_r2, self.dpt_r2, self.confimap_2, self.var_2, self.cf)
        self.confi3_cost = confiStrLoss(self.infer, self.dpt_r3, self.confimap_3, self.var_3, self.cf)
        #self.confi4_cost = confiLoss(self.infer, self.dpt_r, self.confimap_4, self.var_4, self.cf)
        self.ccc_cost = CCC_img(self.infer, self.dpt_r3, self.cf)
        
        # weight decay
        self._vars = tf.trainable_variables()
        self.regul = tf.add_n([tf.nn.l2_loss(v) for v in self._vars if 's1' in v.name]) * 0.00001

        # TOTAL LOSS
        self.confis_avg_cost = (self.confi1_cost + self.confi2_cost + self.confi3_cost)/3
        self.Loss =self.regul  + self.l2_cost*0.3 + self.ccc_cost*0.7 + self.confis_avg_cost*0.3

        # vgg_var
        self.restore_vgg_var = slim.get_variables_to_restore(include=['vgg_16/conv1', 'vgg_16/conv2', 'vgg_16/conv3'])

        # Optimizer
        self._train_op = tf.train.MomentumOptimizer(5e-4, self.cf.momentum)
        self.train_var = bring_var_include_prefix('s1') # + bring_var_include_prefix('vgg_16')
        self.train_op = slim.learning.create_train_op(self.Loss, self._train_op, update_ops=self.train_var)  # option :  update_ops = train_var
        self.mv_avg = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        self.updates = tf.group(*self.mv_avg)
        self.Loss = control_flow_ops.with_dependencies([self.updates], self.Loss)

        # setting
        self.last_e = tf.Variable(0)
        
        # placeholder for metric
        self._targets = tf.placeholder(tf.float32, [None, self.cf.height, self.cf.width, 1])
        self.targets = tf.image.resize_images(self._targets, [120, 160])
        self.predictions = tf.placeholder(tf.float32, [None, 120, 160, 1])
        
        ###METRIC###
        self.absRel = AbsRel_img(self.predictions, self.targets)
        self.sqrRel = SqrRel_img(self.predictions, self.targets)
        self.rmse = RMSE_img(self.predictions, self.targets)
        self.ccc = CCC_img(self.predictions, self.targets, self.cf)
        self.str = strLoss(self.predictions , self.targets, self.cf)
        self.l2 = L2_img(self.predictions, self.targets)
        
        # sess.run list
        self.metric_var = [self.absRel, self.sqrRel, self.rmse, self.ccc, self.str, self.l2]
        self.opt_var = [self.train_op, self.Loss, self.infer, self.confis_avg_cost, self.confi1_cost, self.confi2_cost, self.confi3_cost, self.r_var_1, self.r_var_2, self.r_var_3]
        self.test_var = [self.Loss, self.infer, self.confis_avg_cost, self.confi1_cost, self.confi2_cost, self.confi3_cost, self.r_var_1, self.r_var_2, self.r_var_3,
                         self.infer_size, self.confimap_1_size, self.confimap_2_size, self.confimap_3_size]
        
       
    def confimap(self, confi):
        try:
            size = confi.get_shape().as_list()
        except:
            size = confi.shape
        b = self.cf.batch_size
        h = size[1]
        w = size[2]
        n = h*w # size of image pixels 
        
        _confi = tf.square(confi)
        
        m = tf.reduce_mean(_confi, axis=(1,2))  # mean
        temp=(tf.reshape(_confi, (b, n))) / m
        
        return tf.reshape(temp, (b, h, w, 1))
              
    def vgg(self, inputs, scope='vgg_16'):
        with tf.variable_scope('vgg_16') as sc:
            with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d]):
                net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
                net = slim.max_pool2d(net, [2, 2], scope='pool1')                                   #1/2   (160x120)
                net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
                net = slim.max_pool2d(net, [2, 2], scope='pool2')                                  #1/4   (80x60)
                net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')  
                net = slim.max_pool2d(net, [2, 2], scope='pool3')                                 #1/8   (40x30)
                net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
                return net        
        
    def network(self, vgg, img, reuse=None, is_train=None):
        with tf.variable_scope('s1') as scope:
            if self.cf.reuse:
                scope.reuse_variables()
            w=self.cf.width
            h=self.cf.height
            p=self.cf.padding
            w_init = tf.random_normal_initializer(0., 0.02)
            
            with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, weights_initializer=w_init, normalizer_fn=slim.batch_norm):
                with slim.arg_scope([slim.batch_norm], decay=0.9, center=True, scale=True, updates_collections=None, is_training = is_train): 
                    
                    
                    # image
                    img1 = tf.image.resize_images(img, [int(cf.height/4), int(cf.width/4)])
                    img2 = tf.image.resize_images(img, [int(cf.height/2), int(cf.width/2)])
                    
                    net = slim.repeat(vgg, 6, slim.conv2d, 512, [3, 3], scope='size1')
                    cv1 = slim.conv2d(net, 256, [1, 1], scope='cv1')
                    
                    resize1 = tf.image.resize_images(cv1, [int(cf.height/4), int(cf.width/4)])                                        
                    img_feat1 = slim.repeat(img1, 3, slim.conv2d, 64, [3, 3], scope='img_feat1')
                    concat1 = tf.concat([img_feat1, resize1], axis=3)
                    
                    net = slim.repeat(concat1, 2, slim.conv2d, 128, [3, 3], scope='size2')
                    net = slim.repeat(net, 2, slim.conv2d, 256, [3, 3], scope='size2_')
                    net = slim.repeat(net, 2, slim.conv2d, 384, [3, 3], scope='size2__')
                    cv2 = slim.conv2d(net, 128, [1, 1], scope='cv2')
                    
                    resize2 = tf.image.resize_images(cv2, [int(cf.height/2), int(cf.width/2)])
                    img_feat2 = slim.repeat(img2, 3, slim.conv2d, 64, [3, 3], scope='img_feat2')
                    concat2 = tf.concat([img_feat2, resize2], axis=3)
                    
                    net = slim.repeat(concat2, 4, slim.conv2d, 128, [3, 3], scope='size3')
                    cv3 = slim.conv2d(net, 64, [1, 1], scope='cv3')
                    
                    out = slim.conv2d(cv3, 1, [3, 3], activation_fn=None, scope='out')
        
                    confi1 = slim.conv2d(cv1, 1, [3, 3], padding='VALID', activation_fn=None, normalizer_fn=None, scope='confi1')
                    confi2 = slim.conv2d(cv2, 1, [3, 3], padding='VALID', activation_fn=None, normalizer_fn=None, scope='confi2')
                    confi3 = slim.conv2d(cv3, 1, [3, 3], padding='VALID', activation_fn=None, normalizer_fn=None, scope='confi3')
                                        
                    return out, confi1, confi2, confi3
    
    def training(self):
        # training data and test data.        
        self.img_batch, self.dpt_batch = read.raw_resized_inputs(self.cf.data_dir, self.cf.batch_size)
        self.test_in, self.target = read.test_inputs(self.cf.data_dir, self.cf.batch_size)       
        
        # placeholder for visuailizing
        self.target_img = self.target #tf.image.resize_images(self.target, [int(cf.height/1), int(cf.width/1)]) # target depth image (in TEST time)
        
        # setting
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        global_step = 0
        
        # vgg load
        restore = tf.train.Saver(self.restore_vgg_var)
        restore.restore(sess, os.path.join(cf.vgg_dir, 'vgg_16.ckpt'))
        print ("restore vgg weights")
        
        #saver
        saver = tf.train.Saver(max_to_keep=20)
        ckpt_check_and_load('./ckpt_dir/fuck/confi/paper/', saver, sess)
        #load('./ckpt_dir/fuck/confi/fully/' ,saver, sess)
        #saver.restore(sess, './ckpt_dir/fuck/confi/paper/s1_weights-5929')
        
        # ready
        start_from = sess.run(self.last_e)

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)
        
        # training start
        try:
            while not coord.should_stop():
                for epoch in range(start_from, 1001):
                    #print ("start epoch:", epoch)            
                    total_batch = int(self.cf.num_examples / self.cf.batch_size)
                    absrel=0; sqrrel=0; rmse=0; ccc=0; l2=0; loss=0; structure=0;  # METRIC
                    confi1=0; var1=0; confi2=0; var2=0; confi3=0; var3=0; # CONFIS
                    loss=0; confi_avg_loss=0;  # TOTAL LOSS
                    
                    for i in range(total_batch):
                        #random crop img
                        imgs, dpts = sess.run([self.img_batch, self.dpt_batch])
                        imgs=np.array(imgs, dtype=np.float32); dpts=np.array(dpts, dtype=np.float32) # type converting
                        #noises = np.random.normal(size=[self.cf.batch_size, int(self.cf.height/8)-2, int(self.cf.width/8)-2, 1], loc=0., scale=0.1)
                        
                        _, losses, infers, confis_avg_loss, confis1, confis2, confis3, vars1, vars2, vars3 = sess.run(self.opt_var, feed_dict={self.img_b:imgs, self.dpt_b:dpts,  self.is_train:True})  # learn
                        
                        absrel_c, sqrrel_c, rmse_c, ccc_c, str_c, l2_c = sess.run(self.metric_var, feed_dict={self._targets:dpts, self.predictions:infers}) # metric

                        global_step +=1  # global count

                        #CALCULATE
                        loss+=losses/total_batch; confi_avg_loss+=confis_avg_loss/total_batch # TOTAL LOSS
                        confi1+=confis1/total_batch; confi2+=confis2/total_batch; confi3+=confis3/total_batch;  # CONFIS
                        var1+=vars1/total_batch; var2+=vars2/total_batch; var3+=vars3/total_batch;  # VARS
                        ccc+=ccc_c/total_batch; absrel+=absrel_c/total_batch; sqrrel+=sqrrel_c/total_batch; rmse+=rmse_c/total_batch; structure+=str_c/total_batch; l2+=l2_c/total_batch # METRICS                

                    if epoch==0 or epoch % 10 == 0 :
                        print ("")
                        print ("epoch:", epoch, "loss: %.3f, ccc: %.3f, absrel: %.3f, sqrrel: %.3f, rmse: %.3f, str1:%.3f, l2: %.3f"
                              % (loss, ccc,  absrel, sqrrel, rmse, structure, l2))
                        print ("[AVG_CONFI] : %.3f" % (confi_avg_loss))
                        print ("[[confis] (1): %.3f, (2): %.3f, (3): %.3f, [<vars>] (1): %.3f, (2): %.3f, (3): %.3f " % (confi1, confi2, confi3, var1, var2, var3 ))

                    if epoch==0 or epoch % 30 ==0:
                        save('./ckpt_dir/fuck/confi/paper', 's1_weights', saver, sess, global_step) # save
                        self.test(saver, sess, self.cf, is_training = False) # testr

                    sess.run(self.last_e.assign(epoch + 1)) # up-counting
                break
        except tf.errors.OutOfRangeError:
            print ("oorE")
        finally:
            print ("Finally")
            coord.request_stop()        
        coord.join(threads)
        sess.close()
        
    def test(self, saver, sess, config, is_training):
        # loading
        load('./ckpt_dir/fuck/confi/paper/', saver, sess)

        total_batch = int(self.cf.num_examples_test / self.cf.batch_size)
        absrel=0; sqrrel=0; rmse=0; ccc=0; l2=0; loss=0; structure=0;  # METRIC
        confi1=0; var1=0; confi2=0; var2=0; confi3=0; var3=0; # CONFIS
        loss=0; confi_avg_loss=0;  # TOTAL LOSS

        for i in range(total_batch):
            # input, dpts(=gt_d), binarys(=gt_b), reshaped_dpt(for visual), reshaped_bi(for visual)
            imgs, dpts, tar_imgs = sess.run([self.test_in, self.target, self.target_img]) 
            imgs=np.array(imgs, dtype=np.float32); dpts=np.array(dpts, dtype=np.float32)  # test input  ; # depth target    
            
            #noises= np.random.normal(size=[self.cf.batch_size, int(self.cf.height/8)-2, int(self.cf.width/8)-2, 1], loc=0., scale=0.1)
            
            losses, infers, confis_avg_loss, confis1, confis2, confis3,  vars1, vars2, vars3, infer_sizes, confi1_sizes, confi2_sizes, confi3_sizes = sess.run(self.test_var, feed_dict={self.img_b:imgs, self.dpt_b:dpts, self.is_train:False})  # learn
            
            absrel_c, sqrrel_c, rmse_c, ccc_c, str_c, l2_c = sess.run(self.metric_var, feed_dict={self._targets:dpts, self.predictions:infers}) # metric

            #CALCULATE
            loss+=losses/total_batch; confi_avg_loss+=confis_avg_loss/total_batch # TOTAL LOSS
            confi1+=confis1/total_batch; confi2+=confis2/total_batch; confi3+=confis3/total_batch;# CONFIS
            var1+=vars1/total_batch; var2+=vars2/total_batch; var3+=vars3/total_batch; # VARS
            ccc+=ccc_c/total_batch; absrel+=absrel_c/total_batch; sqrrel+=sqrrel_c/total_batch; rmse+=rmse_c/total_batch; structure+=str_c/total_batch; l2+=l2_c/total_batch # METRICS      

            if i == total_batch-1:
                print ("")
                print ("<TEST> loss: %.3f, ccc: %.3f, absrel: %.3f, sqrrel: %.3f, rmse: %.3f, str1:%.3f, l2: %.3f"
                      % (loss, ccc,  absrel, sqrrel, rmse, structure, l2))
                print ("[AVG_CONFI] : %.3f" % (confi_avg_loss))
                print ("[confis] (1): %.3f, (2): %.3f, (3): %.3f, [<vars>] (1): %.3f, (2): %.3f, (3): %.3f " % (confi1, confi2, confi3,  var1, var2, var3))
            

         # save_img
            if i % 30 ==0:
                b=infer_sizes.shape[0]

                #reshape (esti, tar)
                out_s = np.reshape(infer_sizes,  (b, int(self.cf.height/1), int(self.cf.width/1))) # infer
                tar_d = np.reshape(tar_imgs, (b, int(self.cf.height/1), int(self.cf.width/1)))  # depth target
                cf1 = np.reshape(confi1_sizes, (b, int(self.cf.height/4), int(self.cf.width/4)))
                cf2 = np.reshape(confi2_sizes, (b, int(self.cf.height/2), int(self.cf.width/2)))
                cf3 = np.reshape(confi3_sizes, (b, int(self.cf.height/1), int(self.cf.width/1)))

                # save
                l_epch = str(sess.run(self.last_e))
                i=str(i)
                for img_idx in range(b):
                    idx=str(img_idx)
                    plt.imsave('./img_save/during_train/fuck/paper/temp/'+'['+l_epch+'_'+i+']_'+idx+'_Input.jpg', imgs[img_idx])
                    plt.imsave('./img_save/during_train/fuck/paper/temp/'+'['+l_epch+'_'+i+']_'+idx+'_D_target.jpg', tar_d[img_idx])
                    plt.imsave('./img_save/during_train/fuck/paper/temp/'+'['+l_epch+'_'+i+']_'+idx+'_D_infer.jpg', out_s[img_idx])
                    plt.imsave('./img_save/during_train/fuck/paper/temp/'+'['+l_epch+'_'+i+']_'+idx+'_C_1.jpg', cf1[img_idx])
                    plt.imsave('./img_save/during_train/fuck/paper/temp/'+'['+l_epch+'_'+i+']_'+idx+'_C_2.jpg', cf2[img_idx])
                    plt.imsave('./img_save/during_train/fuck/paper/temp/'+'['+l_epch+'_'+i+']_'+idx+'_C_3.jpg', cf3[img_idx])
                    
    def load_weights(self, sess, name):
        
        restore_enc_var = slim.get_variables_to_restore(include=['s1/'])
        restore_enc = tf.train.Saver(restore_enc_var)
        restore_enc.restore(sess, name)
        
        print ("restore 's1' weights")

In [6]:
class config(object):
    def __init__(self):
        self.vgg_dir = './vgg_w'
        self.data_dir='data_read/data/fcv'
        self.batch_size= 2
        self.is_training=True
        self.reuse=None
        self.gan=True
        self.height=240
        self.width=320
        self.padding='SAME'
        self.momentum=0.9 # 이게 문제가 될수도..
        self.epoch_size =501
        self.num_examples=(795)
        self.num_examples_test=654
        self.ckpt_dir ='./ckpt_dir'
        self.state='s1'

In [7]:
cf=config()
CN = Confi_Net(cf)

In [8]:
CN.training()  # 이젠 포기다

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 5.339, ccc: 0.999, absrel: 0.471, sqrrel: 0.923, rmse: 1.572, str1:1.399, l2: 3.832
[AVG_CONFI] : 4.127
[[confis] (1): 7.362, (2): 3.372, (3): 1.647, [<vars>] (1): 0.281, (2): 0.109, (3): 0.186 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 18155264.691, ccc: 1.000, absrel: 1998.138, sqrrel: 11317324.716, rmse: 5534.624, str1:2487350.352, l2: 30632062.391
[AVG_CONFI] : 5678465.067
[confis] (1): 13639693.346, (2): 2571313.890, (3): 824387.908, [<vars>] (1): 0.040, (2): 0.124, (3): 0.125 

epoch: 10 loss: 3.939, ccc: 0.999, absrel: 0.475, sqrrel: 0.747, rmse: 1.301, str1:1.108, l2: 2.252
[AVG_CONFI] : 2.848
[[confis] (1): 5.401, (2): 2.

KeyboardInterrupt: 

In [ ]:
CN.training()  # 연결고리 이어서

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397
Successfully loaded: ./ckpt_dir/fuck/confi/paper/s1_weights-397

epoch: 150 loss: 2.742, ccc: 0.867, absrel: 0.460, sqrrel: 0.702, rmse: 1.249, str1:1.127, l2: 2.051
[AVG_CONFI] : 1.992
[[confis] (1): 3.800, (2): 1.471, (3): 0.706, [<vars>] (1): 0.896, (2): 0.505, (3): 0.222 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 2.441, ccc: 0.801, absrel: 0.326, sqrrel: 0.460, rmse: 1.146, str1:1.156, l2: 1.638
[AVG_CONFI] : 1.869
[confis] (1): 3.609, (2): 1.354, (3): 0.645, [<vars>] (1): 0.848, (2): 0.463, (3): 0.223 

epoch: 160 loss: 2.701, ccc: 0.869, absrel: 0.439, sqrrel: 0.665, rmse: 1.230, str1:1.095,

In [ ]:
CN.training()  # 연결고리

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 4.995, ccc: 1.000, absrel: 0.487, sqrrel: 1.066, rmse: 1.750, str1:1.305, l2: 4.700
[AVG_CONFI] : 3.707
[[confis] (1): 6.400, (2): 3.232, (3): 1.490, [<vars>] (1): 0.332, (2): 0.182, (3): 0.122 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 4.028, ccc: 0.996, absrel: 0.334, sqrrel: 0.499, rmse: 1.260, str1:1.160, l2: 2.029
[AVG_CONFI] : 4.443
[confis] (1): 5.442, (2): 6.586, (3): 1.301, [<vars>] (1): 0.942, (2): 4.263, (3): 0.485 

epoch: 10 loss: 3.272, ccc: 0.999, absrel: 0.480, sqrrel: 0.754, rmse: 1.322, str1:1.138, l2: 2.318
[AVG_CONFI] : 2.639
[[confis] (1): 5.105, (2): 1.981, (3): 0.832, [<vars>] (1): 0.507, (2): 0.467, (3): 0.

In [5]:
Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 4.968, ccc: 0.987, absrel: 0.484, sqrrel: 1.056, rmse: 1.738, str1:1.387, l2: 4.629
[AVG_CONFI] : 3.590
[[confis] (1): 6.424, (2): 2.864, (3): 1.481, [<vars>] (1): 0.655, (2): 0.328, (3): 0.156 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 3.770, ccc: 0.944, absrel: 0.373, sqrrel: 0.632, rmse: 1.335, str1:1.189, l2: 2.162
[AVG_CONFI] : 3.703
[confis] (1): 6.029, (2): 2.475, (3): 2.605, [<vars>] (1): 0.405, (2): 0.402, (3): 1.479 

epoch: 10 loss: 3.103, ccc: 0.910, absrel: 0.470, sqrrel: 0.739, rmse: 1.293, str1:1.213, l2: 2.228
[AVG_CONFI] : 2.338
[[confis] (1): 4.233, (2): 1.856, (3): 0.926, [<vars>] (1): 0.977, (2): 0.517, (3): 0.203 

epoch: 20 loss: 2.788, ccc: 0.884, absrel: 0.448, sqrrel: 0.679, rmse: 1.238, str1:1.121, l2: 2.001
[AVG_CONFI] : 1.963
[[confis] (1): 3.475, (2): 1.600, (3): 0.814, [<vars>] (1): 1.024, (2): 0.508, (3): 0.209 

epoch: 30 loss: 2.684, ccc: 0.847, absrel: 0.436, sqrrel: 0.653, rmse: 1.223, str1:1.075, l2: 1.978
[AVG_CONFI] : 1.816
[[confis] (1): 3.228, (2): 1.471, (3): 0.750, [<vars>] (1): 1.034, (2): 0.486, (3): 0.208 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-12307

<TEST> loss: 2.425, ccc: 0.797, absrel: 0.308, sqrrel: 0.425, rmse: 1.133, str1:1.151, l2: 1.604
[AVG_CONFI] : 1.721
[confis] (1): 2.990, (2): 1.439, (3): 0.735, [<vars>] (1): 0.894, (2): 0.381, (3): 0.159 
CN.training()  # 호잉?

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-12307
Successfully loaded: ./ckpt_dir/fuck/confi/paper/s1_weights-12307

epoch: 30 loss: 2.897, ccc: 0.847, absrel: 0.440, sqrrel: 0.691, rmse: 1.275, str1:1.202, l2: 2.193
[AVG_CONFI] : 2.027
[[confis] (1): 3.609, (2): 1.639, (3): 0.832, [<vars>] (1): 1.042, (2): 0.502, (3): 0.217 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 2.448, ccc: 0.784, absrel: 0.319, sqrrel: 0.447, rmse: 1.129, str1:1.180, l2: 1.581
[AVG_CONFI] : 1.805
[confis] (1): 3.190, (2): 1.489, (3): 0.734, [<vars>] (1): 1.187, (2): 0.495, (3): 0.189 

epoch: 40 loss: 2.678, ccc: 0.821, absrel: 0.434, sqrrel: 0.664, rmse: 1.216, str1:1.10

KeyboardInterrupt: 

In [ ]:
CN.training()  # 호잉?

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 4.968, ccc: 0.987, absrel: 0.484, sqrrel: 1.056, rmse: 1.738, str1:1.387, l2: 4.629
[AVG_CONFI] : 3.590
[[confis] (1): 6.424, (2): 2.864, (3): 1.481, [<vars>] (1): 0.655, (2): 0.328, (3): 0.156 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 3.770, ccc: 0.944, absrel: 0.373, sqrrel: 0.632, rmse: 1.335, str1:1.189, l2: 2.162
[AVG_CONFI] : 3.703
[confis] (1): 6.029, (2): 2.475, (3): 2.605, [<vars>] (1): 0.405, (2): 0.402, (3): 1.479 

epoch: 10 loss: 3.103, ccc: 0.910, absrel: 0.470, sqrrel: 0.739, rmse: 1.293, str1:1.213, l2: 2.228
[AVG_CONFI] : 2.338
[[confis] (1): 4.233, (2): 1.856, (3): 0.926, [<vars>] (1): 0.977, (2): 0.517, (3): 0.

In [5]:
CN.training()   # 124

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 4.768, ccc: 0.984, absrel: 0.490, sqrrel: 1.070, rmse: 1.745, str1:1.310, l2: 4.623
[AVG_CONFI] : 3.207
[[confis] (1): 5.467, (2): 2.768, (3): 1.386, [<vars>] (1): 0.743, (2): 0.314, (3): 0.154 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 3.056, ccc: 0.965, absrel: 0.331, sqrrel: 0.487, rmse: 1.253, str1:1.092, l2: 2.004
[AVG_CONFI] : 2.420
[confis] (1): 4.165, (2): 2.068, (3): 1.028, [<vars>] (1): 0.564, (2): 0.263, (3): 0.095 
Finally


KeyboardInterrupt: 

In [5]:
CN.training()  

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 4.302, ccc: 0.973, absrel: 0.481, sqrrel: 1.059, rmse: 1.731, str1:1.300, l2: 4.650
[AVG_CONFI] : 2.175
[[confis] (1): 9.950, (2): 2.818, (3): 1.532, [<vars>] (1): 3.372, (2): 0.278, (3): 0.139 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 2.751, ccc: 0.937, absrel: 0.340, sqrrel: 0.507, rmse: 1.241, str1:1.128, l2: 1.905
[AVG_CONFI] : 1.853
[confis] (1): 10.120, (2): 2.481, (3): 1.225, [<vars>] (1): 3.860, (2): 0.626, (3): 0.211 

epoch: 10 loss: 2.575, ccc: 0.909, absrel: 0.453, sqrrel: 0.701, rmse: 1.259, str1:1.147, l2: 2.124
[AVG_CONFI] : 1.311
[[confis] (1): 11.349, (2): 1.772, (3): 0.850, [<vars>] (1): 4.909, (2): 0.514, (3): 

KeyboardInterrupt: 

In [5]:
CN.training()  

Filling queue with 318 Images before starting to train. This will take a few minutes.
Filling queue with 318 Images before starting to train. This will take a few minutes.
INFO:tensorflow:Restoring parameters from ./vgg_w/vgg_16.ckpt
restore vgg weights
Could not find old network weights

epoch: 0 loss: 4.895, ccc: 0.982, absrel: 0.487, sqrrel: 1.064, rmse: 1.746, str1:1.332, l2: 4.611
[AVG_CONFI] : 3.389
[[confis] (1): 5.879, (2): 2.856, (3): 1.432, [<vars>] (1): 0.699, (2): 0.391, (3): 0.143 
INFO:tensorflow:Restoring parameters from ./ckpt_dir/fuck/confi/paper/s1_weights-397

<TEST> loss: 3.271, ccc: 1.005, absrel: 0.342, sqrrel: 0.524, rmse: 1.305, str1:1.157, l2: 2.157
[AVG_CONFI] : 2.573
[confis] (1): 4.273, (2): 2.330, (3): 1.116, [<vars>] (1): 0.948, (2): 0.429, (3): 0.114 
Finally


KeyboardInterrupt: 